In [ ]:
# lecture 2 code
using PyPlot
using BenchmarkTools

In [ ]:
#求解上三角矩阵
function back_sub(A, b)
    x = zeros(n)
    for i = n:-1:1
        x[i] = b[i]
        for j = i + 1 : n
            x[i] -= A[i, j] * x[j]
        end
        x[i] = x[i] / A[i, i]
    end
    return x
end

In [ ]:
#高斯消元法
function linear_solver(A, b)
    n = length(b)

    for row = 1:n
        # println(row)
        for i = row + 1 : n
            # println(i)
            coef = A[i, row] / A[row, row]
            for j = row : n
                A[i, j] = A[i, j] - coef * A[row, j]
                # println(j)
                
            end
            # 课上的程序缺少了这行
            b[i] = b[i] - coef * b[row]
        end
        # println(A)
    end
    return back_sub(A,b)
end

In [ ]:
#修改高斯消元法(不带排序，把A和b合并处理)
function modify_linear_solver(A, b)
    n = length(b)
    C =hcat(A,b)
    # println(C)
    for row = 1:n
        # println("row = ",row)
        for i = row + 1 : n
            # println("i = ",i)
            coef = C[i, row] / C[row, row]
            for j = row : n+1
                # println("j = ",j)
                C[i, j] = C[i, j] - coef * C[row, j]
            end
        end     
        # println(C)
    end
    D = C[:,1:n]
    e = C[:,n+1]
    # println(D)
    # println(e)
    return back_sub(D,e)
end

In [ ]:
#随机生成n阶矩阵
n = 100
A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);

In [ ]:
modify_linear_solver(A, b)
@btime modify_linear_solver(A, b)

In [ ]:
linear_solver(A, b)
@btime linear_solver(A, b)

In [ ]:
A \ b
@btime A\b

In [ ]:
#带排序的高斯消元法（列主元）
function sort_gauss_elimination(A, b)
    n = length(b)

    #排序
    for k = 1 : n-1  #从1到n-1列，最后一列不需要排序，因为就一个
        row_max = k #默认第k行k列是最大的
        for m = k+1 :n  #从k+1到n行
            #找最大行的位置
            if abs(A[m,k]) > abs(A[row_max,k])
                row_max = m
            end
        end

        #判断是否需要交换
        if row_max != k #需要交换
            # 使用临时变量temp_row来存储第k行的内容
            temp_row = A[k, :]
            temp_b = b[k]
            
            # 将第max_row行的内容复制到第k行
            A[k, :] = A[row_max, :]
            b[k] = b[row_max]
            
            # 将之前第k行（现在存储在temp变量中）的内容复制到第max_row行
            A[row_max, :] = temp_row
            b[row_max] = temp_b
        end

    end

    #消元，化上三角
    for row = 1:n
        for i = row + 1 : n
            coef = A[i, row] / A[row, row]
            for j = row : n
                A[i, j] = A[i, j] - coef * A[row, j]
            end
            b[i] = b[i] - coef * b[row]
        end
    end
    return back_sub(A,b)
end

In [ ]:
@btime sort_gauss_elimination(A, b)

In [ ]:
@btime A\b

In [ ]:
#测试算例,随机生成n阶矩阵
n = 20
A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);